# From Movie Reviews to Box Office
Yen-Ting Chen

## Table of Contents
[Overview](#overview)  
[Data Scraping and Cleaning](#data_scraping)  
[Feature Extraction](#feature_extraction)  
[Prediction Modeling](#prediction_modeling)  
[Discussion and Conclusion](#discussion_conclusion)  
[References](#references)  

<a id='overview'></a>
## Overview
By granting us the ability to share product reviews, Internet benefits not only customers for understanding the products of interest, but also businesses for addressing potential improvements. It is well known that customer reviews are a gold mine of information, about the future of the products and business directions. The movie industry is one of the industries that can greatly benefit from such information mining. Some studies have explored the possibility of predicting the opening weekend box office.<sup>1-3</sup> This project aims to explore the correlation between movie box office and Amazon movie reviews, and develop predictive models for projecting the overall movie box office.

There are three main parts to this project: 1) data scraping and cleaning, 2) feature extraction, and 3) prediction modeling. Parts of the codes used are shown here, while the rest can be accessed on my <a href="https://github.com/janie128/Project-movies" target="_blank">Github repository</a>.  

![](./figures/flowchart.png)

<a id='data_scraping'></a>
## Data Scraping and Cleaning
There are two main data sets used in this project. One is the data set of movies and their box offices scraped from the Box Office Mojo website,<sup>4</sup> and the second is Amazon review data.<sup>5-7</sup> 

#### Box office data
Box office data was scraped from the website Box Office Mojo for all years and movies. Css selectors for the pertinent information fields were found with the SelectorGadget Chrome extension.

In [ ]:
# The following URL and Css templates rely on the layout of the website. They may change over time. 
pageUrlTemplate <- "http://www.boxofficemojo.com/daily/?view=bymovie&yr=%s&page=%d&sort=title&order=ASC"
years <- c("2015", "2014", "2013", "2012", "2011",
          "2010", "2009", "2008", "2007", "2006",
          "2005", "2004", "2003", "2002", "pre2002")
remainingPageLinkCss <- "font:nth-child(4) a" # how many page remaining
movieTitleCss <- "tr+ tr td:nth-child(1) font" # movie title
releaseGrossCss <- "tr+ tr td:nth-child(4) font" # release gross
releaseDateCss <- "tr+ tr td:nth-child(5) font" # release date

# Function for parsing the html page and extracting data of interest.
extractBoxOfficeFn <- function(page, boxOffice) {
  extractedTitles <- html_text(html_nodes(page, movieTitleCss))
  extractedReleaseDates <- html_text(html_nodes(page, releaseDateCss))
  # Extract release gross, strip "$" and ",", and cast to numeric.
  extractedReleaseGrosses <- html_text(html_nodes(page, releaseGrossCss))
  extractedReleaseGrosses <- gsub("\\$", "", extractedReleaseGrosses)
  extractedReleaseGrosses <- gsub(",", "", extractedReleaseGrosses)
  extractedReleaseGrosses <- as.numeric(extractedReleaseGrosses)

  extractedFromPage <- data.frame(
      title = extractedTitles,
      gross = extractedReleaseGrosses,
      releaseDate = extractedReleaseDates)
  return(extractedFromPage)
}

for (year in years) {
  pageUrl <- sprintf(pageUrlTemplate, year, 1);
  page <- read_html(pageUrl)
  numRemainingPages <- length(html_text(html_nodes(page, remainingPageLinkCss)))

  boxOffice <- rbind(boxOffice, extractBoxOfficeFn(page, boxOffice))
  for (pageIndex in 1:numRemainingPages) {
    pageUrl <- sprintf(pageUrlTemplate, year, pageIndex + 1);
    page <- read_html(pageUrl)
    boxOffice <- rbind(boxOffice, extractBoxOfficeFn(page, boxOffice))
  }
}

The data was then filtered to include data only after May 1996, to match the duration period of the Amazon review data. Initially, the box office amounts were to be adjusted to account for inflation or cultural factors, such as higher movie-going culture over the years. This could be done by fitting a general trend regression analysis by year and factoring it out. However, over the time period, an increase in the percentage of small films (lower-grossing) disrupts any such trend. Figure 1 illustrates this increase, by showing the percentage of movies grouped by their box office logarithm amounts over the years.  

![](./figures/box_office_years.png)
<h4 align="center">Fig.1 Box office ($log_{10}(gross)$) sorted into 4 buckets and plotted over years. Percentage of highest grossing movies decreases while medium and lower grossing movies are on the rise.</h4>

Therefore, inflation rate was the only factor for adjustment of the gross amounts. US inflation rates from 1995-2014 were obtained from the website US Inflation Calculator<sup>8</sup> and applied to the data.

#### Amazon movie review data
Millions of Amazon "Movies & TV" review data and their metadata files were downloaded from Dr. McAuley's webpage<sup>5</sup> in JSON format. This was further parsed and output with a Python script into strict JSON format for parsing in R. However, review/metadata files were labeled with product titles as opposed to specifically movie titles. For example, the reviews for the movie "Jurassic Park" can be found under the product title "Laserdisc - Jurassic Park - Letterboxed". Thus, matching between the movie titles from the box office data set and the product titles available in the metadata file was required. The data matching process involved stripping punctuation, converting to lower case, removing leading "the" for a more comprehensive matching, and pattern matching with regular expressions.  
  
The data was further cleaned and reviews with unmatched titles removed. As some mismatching was unavoidable with extremely short titles, a subset of this data where the number of characters was greater than 4 was used for the rest of the analysis.

<a id='feature_extraction'></a>
## Feature Extraction
Raw variables of interest in the review data include the movie reviews and overall rating for each review. Further meaningful features must be extracted from this data. 

**(1) Review count (by movie)**: Figure 2 shows a histogram of the distribution of review counts for all movies. As can be seen, there is a large proportion of movies with review counts below 50 (red line). However, these are considered to be too few to be representative of review quality and are thus discarded for the analysis.  
![](./figures/review_count.png)
<h4 align="center">Fig.2 Distribution of review counts per movie. Data below counts of 50 (red line) are discarded.</h4>
  
**(2) Ratings distribution**: Figure 3 shows three movie samples and their ratings distribution. The sample plots show that ratings distributions are significantly different for each movie, and the average of the rating alone is not sufficient. Therefore, rating scores of 1 through 5 and their frequencies are extracted as features.  
![](./figures/review_ratings.png)
<h4 align="center">Fig.3 Three samples of movie ratings distribution.</h4>

**(3) Average word count, word count percentiles**: Word count for each review was calculated using regex search, then aggregated for each movie. Word count distributions for a sample of three movies are plotted in Figure 4, and show that the distributions are quite different. Low, mid and high percentiles of the word count distribution for each movie were therefore extracted as features in addition to the average.  
![](./figures/word_distribution.png)
<h4 align="center">Fig.4 Three samples of word count distribution.</h4>

In [ ]:
# ---------Reviews word count
words <- as.data.frame(sapply(gregexpr("\\S+", reviewsTitles$reviewText), length))
colnames(words) <- "wordCount"
reviewsTitles <- cbind(reviewsTitles, words)

# ---------Generate table with review word count info including quantiles and average
wordCountInfo <- reviewsTitles %>% group_by(movieTitle) %>%
  summarize(wordCountLow = quantile(wordCount, probs=0.25), wordCountMid = quantile(wordCount, probs=0.5), 
            wordCountHigh = quantile(wordCount, probs=0.75), wordCountAvg = round(mean(wordCount),1))

**(4) TF-IDF of good/bad words (review content analysis)**: The review contents were analyzed with natural language processing techniques including tokenization, and sentiment analysis was performed to produce a set of "good" (positive sentiment) and "bad" (negative sentiment) term frequency-inverse document frequencies (TF-IDF).  

![](./figures/TFIDF_flowchart.png)

The tm package was used to perform text mining on each review to obtain a list of occurring words and their frequencies. The usual text cleaning techniques were conducted, excluding stopwords removal. This was based on consideration of the amount of information it may be removing on relatively short reviews. Dictionaries of good (positive sentiment) and bad (negative sentiment) words were obtained from Hu and Liu's Opinion Lexicon<sup>9-10</sup>. They were compared to the generated occurring words list to find the relevant words.  

IDF for the two dictionaries were calculated according to:  

$$idf(t) = log_{10}\frac{N}{1+ \sum\limits_{d\in D}I(t,d)},\;\;\;\; I(t,d) = \left\{\begin{array}{ll} 0 & \mbox{if}\;\;t\in d \\ 1 & \mbox{if}\;\;t\not\in d \end{array}\right.$$

where $t$ is the term or word in consideration, $N$ is the total number of reviews, $d$ is the review in consideration, and $D$ is the collection of all reviews. The denominator represents the number of reviews where the term $t$ appears. This is essentially a weighting for how common a given word is in the set of reviews: the less common it is, the higher its importance and stronger the weighting. 

Parallel processing with multicores was utilized to speed up the process for the near-millions of review data.

In [ ]:
# -----------------------
# Adding IDF to goodDict
# -----------------------
totalDoc <- length(textAllReviews) # total number of documents

goodDictRegex <- goodDict$regex
goodDictContain <- numeric(length(goodDictRegex))

# Parallel processing & exporting data needed in this segment
numCores <- min(3, detectCores() - 1)
cluster <- makeCluster(numCores)
clusterExport(cluster, "textAllReviews")
# parLapply used instead of lapply for parallel processing

# --Done in chunks to ensure data is saved if anything happens
# Count number of documents(reviews) that contain the good word (loops through goodDict)
limit <- length(goodDictRegex)
begin <- 1
end <- 50

while (begin <= limit){
  realEnd = min(limit, end)
  
  results <- parLapply(cluster, goodDictRegex[begin:realEnd], function(x) {
    sum(grepl(paste("\\<", x, "\\>", sep = ""), textAllReviews))
  })
  goodDictContain[begin:realEnd] <- results
  
  # For monitoring progress
  print(paste(end, " out of ", limit))
 
  begin <- begin + 50
  end <- end + 50
}
goodDict$contain <- goodDictContain
# Turn off parallel
stopCluster(cluster)
rm(cluster)
 
# Calculate inverse document frequency with denominator +1
goodDict$IDF <- lapply(goodDict$contain, function(x) {round(log10(totalDoc/(1 + x)),4)})

Subsequently, the term frequencies for each relevant word in a given review were calculated:  

$$tf(t,d) = \frac{f_{t,d}}{max\{f_{t^{'},d}:\;t^{'}\in d\}}$$

where $f_{t,d}$ is the raw frequency of the term $t$ occuring in review $d$ and $max\{f_{t^{'},d}:\;t^{'}\;\epsilon\;d\}$ is the maximum raw frequency of any term in the given review, which accounts for the differing lengths of the reviews. The term frequency-inverse document frequency (TF-IDF) $tfidf(t,d,D) = tf(t,d)\times idf(t,D)$ is then calculated for each word, and the good words and bad words each summed for the review. This process is repeated for all reviews, then averaged for each movie.

In [ ]:
# A) Function for generating vector of words in each review (return is class char vector)
reviewToWordsFn <- function(text){
  text_source <- VectorSource(text)
  textCorpus <- Corpus(text_source)
  textCorpus <- tm_map(textCorpus, content_transformer(tolower))
  textCorpus <- tm_map(textCorpus, removePunctuation)
  textCorpus <- tm_map(textCorpus, stripWhitespace)
  dtm <- as.matrix(DocumentTermMatrix(textCorpus, control=list(wordLengths=c(1,Inf))))
  return(dtm)
}

# B) Function for generating good OR bad TF-IDF from list of words, returns list of TF-IDF (good or bad)
wordsToSentiTfidfFn <- function(listOfWordsFreq, maxTF, sentiDict){
  listOfSentiWords <- intersect(listOfWordsFreq$words, sentiDict$words) # char vector of words that are good/bad
  listOfSentiWordsTFIDF <- subset(listOfWordsFreq, words %in% listOfSentiWords) # df of good/bad words with their TF
  listOfSentiWordsTFIDF$TF <- round(listOfSentiWordsTFIDF$freq/maxTF, 5) # raw TF of word / total word count in document
  listOfSentiWordsIDF <- subset(sentiDict, words %in% listOfSentiWords)
  listOfSentiWordsTFIDF$IDF <- listOfSentiWordsIDF$IDF
  rm(listOfSentiWordsIDF)
  listOfSentiWordsTFIDF$TFIDF <- mapply(function(x,y) {x*y},
                                        listOfSentiWordsTFIDF$TF, listOfSentiWordsTFIDF$IDF) # TF*IDF
  return(listOfSentiWordsTFIDF)
}

# C) Function for generating total TF-IDF from reviews. Calls functions A & B.
#    Returns a vector with sum of good words and bad words TFIDF (for each input review).
reviewToTfidfFn <- function(text){
  listOfWordsFreq <- reviewToWordsFn(text) # matrix of all words (as colname) with their frequencies
  maxTF <- max(listOfWordsFreq) # numeric, maximum frequency of above matrix
  listOfWords <- colnames(listOfWordsFreq) # char vector of all words
  listOfWordsFreq <- cbind(words=listOfWords, freq=listOfWordsFreq[1,]) # matrix of words & freq
  row.names(listOfWordsFreq) <- NULL
  rm(listOfWords)
  listOfWordsFreq <- as.data.frame(listOfWordsFreq, stringsAsFactors = FALSE) # df of words & freq
  listOfWordsFreq$freq <- as.numeric(listOfWordsFreq$freq)
 
  # call for good words TF-IDF list
  goodTFIDFList <- wordsToSentiTfidfFn(listOfWordsFreq, maxTF, goodDict)
  # call for bad words TF-IDF list
  badTFIDFList <- wordsToSentiTfidfFn(listOfWordsFreq, maxTF, badDict)
  
  goodTFIDF <- ifelse(dim(goodTFIDFList)[1]!=0, sum(goodTFIDFList$TFIDF), 0)
  badTFIDF <- ifelse(dim(badTFIDFList)[1]!=0, sum(badTFIDFList$TFIDF), 0)

  return(c(goodTFIDF, badTFIDF))
}

![](./figures/TFIDF.png)
<h4 align="center">Fig.5 Relation between TF-IDF generated from good (positive sentiment) and bad (negative sentiment) words and review average score.</h4>

In Figure 5, it can be seen from the relation between good and bad words TF-IDF's to review average scores, that the calculated TFIDF's are likely a good indication of the review content.  

<a id='prediction_modeling'></a>
## Prediction Modeling

With the features extracted, three different machine learning models were trained to predict the order of magnitude of the box office gross. The gross was classified into buckets of $0-10^5$, $10^5-10^6$, $10^6-10^7$, $10^7-10^8$ and $10^8-10^9$. The data was partitioned into training (70%) and testing (30%) sets, and the same sets were used for training of the three models. The resampling method (repeated cross-validation) and the specified random seed was held constant in order to acquire deterministic results. The three models trained include a Boosted Logistic Regression Model, a Naive Bayes model, and a Random Forest model.  

Of these, the Random Forest model outperformed the rest, and was further tuned via a grid search on the mtry parameter, the number of variables sampled at each split. Tuning revealed there to be minimal difference between mtry values from 2 to 8, with mtry = 2 having a slightly higher accuracy. The optimization comparison is shown in Figure 6.


In [ ]:
trainCtrl <- trainControl(method = "repeatedCV", number=10, repeats=3)
modelFitTuneRF <- train(buckets ~ ., data=training, method = "rf", prox=TRUE, trainControl=trainCtrl,
                        tuneGrid = expand.grid(mtry = c(2,3,4,5,6,7,8)))

![](./figures/model_tuning.png)
<h4 align="center">Fig.6 Random Forest mtry parameter optimization and accuracy.</h4>

This final model was applied to the testing set and achieved an accuracy of 63.02%. Figure 7 shows the features plotted in order of importance as determined by the Random Forest model. The average TF-IDF for bad words and the number of review counts are among the highest ranked.

![](./figures/model_features.png)
<h4 align="center">Fig.7 Feature importance as determined by the Random Forest model.</h4>

<a id='discussion_conclusion'></a>
## Discussion and Conclusion
There are potentially three main points that could improve the accuracy. The first consideration is the validity of the review data. As discussed in the data cleaning section, Amazon review data names were given for product titles and not movie titles. Steps were taken to minimize mismatches, but some were still unavoidable. The shorter the movie title, the more likely this is to occur, hence the removal of all data where movie title has character count of less than four. A higher threshold could not be implemented without the expense of losing too many data observations. Lowering the mismatch in the titles could improve the correlation in the data.  

A second possibility is the relevance of the review content (both sentiment and length) to the ratings. A review could spend many words elaborating on the actual content of a movie, instead of the reviewer's views on the movie, and thus an analysis would be highly skewed by the sentiment of the movie itself. Adding the movie genre could compensate for the sentiment skewness.

Lastly, is to acquire more related information to extract features from. A possible indicator of box office gross could include the number of movie theatres the movie was released in, or maybe the film's promotional budget. Average review score was, contrary to expectations, not very indicative of how well the movie sold. Examples can be found where a hyped-up high-grossing movie had very low review scores, or vice versa, where a niche movie did not sell well but was much loved by its followers.

The features of most importance were the review count and analysis of the review content. It is rather surprising, however, that the TF-IDF value for good words has much less importance than that of the bad words. The good and bad TF-IDF, word count percentiles, and review rating levels were all extracted as individual features based on the reasons discussed in the feature extraction section. However, despite the reasoning that these features each convey a different level of information, there may be correlation between them and that they are not independent. This may affect the performance of the prediction modeling. Principle component analysis (PCA) could have been applied to alleviate this, but was chosen not to, in order to preserve the interpretability of the model.

In its current form, the prediction system trained in this project has room for improvement. Once additional information can be obtained and features extracted, this can prove a very robust method of projecting a movie's box office gross. Future applications include transforming this analysis to be modeled upon critics' reviews instead, and can thus be used to predict the box office ahead of the movie's official opening date.

<a id='references'></a>
## References
1. <a href="http://blog.ranker.com/predicting-box-office-success-ranker-data/#.VotBL_krKUk" target="_blank">Predicting Box Office Success a Year in Advance from Ranker Data</a>
2. "Predicting the Future with Social Media" S. Asur, B. A. Huberman; *2010 IEEE/WIC/ACM International Conference on Web Intelligence and Intelligent Agent Technology*, 2010, pp. 492-499
3. "Early Prediction of Movie Box Office Success Based on Wikipedia Activity Big Data" M. Mestyan, T. Yasseri, J. Kertesz; *PLoS ONE 8(8): e71226*, 2013
4. <a href="http://www.boxofficemojo.com/daily/?view=bymovie&yr=all&sort=title&order=ASC&p=.htm" target="_blank">Box Office Mojo</a>
5. <a href="http://jmcauley.ucsd.edu/data/amazon/" target="_blank">Dr. Julian McAuley's Amazon product data</a>
6. "Image-based recommendations on styles and substitutes" J. McAuley, C. Targett, J. Shi, A. van den Hengel; *SIGIR*, 2015
7. "Inferring networks of substitutable and complementary products" J. McAuley, R. Pandey, J. Leskovec; *Knowledge Discovery and Data Mining*, 2015
8. <a href="http://www.usinflationcalculator.com/inflation/historical-inflation-rates/" target="_blank">US Inflation Calculator</a>
9. <a href="https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon" target="_blank">Hu and Liu's Opinion Lexicon</a>
10. "Mining and Summarizing Customer Reviews" M. Hu, B. Liu; *Proceedings of the ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD-2004)*, Aug 22-25, 2004, Seattle, Washington, USA